# 컬럼 탐색 및 분류

In [15]:
import pandas as pd
import os

folder_path = os.path.join("..", "data", "raw")

# 파일 경로 설정
train_path = os.path.join(folder_path, "train.csv")
test_path = os.path.join(folder_path, "test.csv")

# 파일 로드
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# 확인
print("✅ train:", train.shape)
print("✅ test:", test.shape)
train.head()


✅ train: (1460, 81)
✅ test: (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
# 전체 데이터 구조 및 타입 확인
train.info()

# 모든 컬럼 요약 통계 (범주형 포함)
train.describe(include='all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,1460.0,NaN,NaN,NaN,730.5,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,NaN,NaN,NaN,56.89726,42.300571,20.0,20.0,50.0,70.0,190.0
MSZoning,1460,5,RL,1151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotFrontage,1201.0,NaN,NaN,NaN,70.049958,24.284752,21.0,59.0,69.0,80.0,313.0
LotArea,1460.0,NaN,NaN,NaN,10516.828082,9981.264932,1300.0,7553.5,9478.5,11601.5,215245.0
...,...,...,...,...,...,...,...,...,...,...,...
MoSold,1460.0,NaN,NaN,NaN,6.321918,2.703626,1.0,5.0,6.0,8.0,12.0
YrSold,1460.0,NaN,NaN,NaN,2007.815753,1.328095,2006.0,2007.0,2008.0,2009.0,2010.0
SaleType,1460,9,WD,1267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SaleCondition,1460,6,Normal,1198,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# 기본 정보 수집
summary = pd.DataFrame({
    'Column': train.columns,
    'Dtype': train.dtypes.values,
    'Missing': train.isnull().sum().values,
    'Unique': train.nunique().values
})

# 범주형 여부 추정 (단순 기준)
def guess_categorical(row):
    if row['Dtype'] == 'object':
        return 'O'
    elif row['Unique'] < 20:
        return 'Possible'
    else:
        return ''

summary['Guess_Categorical'] = summary.apply(guess_categorical, axis=1)

# 보기 좋게 정렬
summary = summary.sort_values(by='Missing', ascending=False).reset_index(drop=True)
summary.head()

# Excel로 저장
output_path = "../data/description/column_summary.xlsx"
summary.to_excel(output_path, index=False)

print("✅ 저장 완료:", output_path)



✅ 저장 완료: ../data/description/column_summary.xlsx


In [ ]:
import pandas as pd
import numpy as np

# 1. 타깃 변수 정의
target_col = 'SalePrice'
total_rows = len(train)

# 2. 결과를 담을 리스트
summary = []

# 3. 컬럼별 루프
for col in train.columns:
    if col == target_col:
        continue

    dtype = train[col].dtype
    nunique = train[col].nunique()
    missing = train[col].isnull().sum()
    missing_ratio = round((missing / total_rows) * 100, 2)
    
    # (1) 유형 분류
    if dtype in ['int64', 'float64']:
        feature_type = '수치형'
    elif dtype == 'object' and nunique <= 20:
        feature_type = '범주형'
    else:
        feature_type = '범주형'
    
    # (2) 순서형 여부 추정
    ordinal = 'O' if any(key in col for key in ['Qual', 'Cond', 'Finish', 'Exposure', 'Slope']) else 'E'
    
    # (3) 인코딩 방식
    if feature_type == '수치형':
        encoding = '스케일링 또는 그대로 사용'
    elif ordinal == 'O':
        encoding = '순서형 인코딩'
    else:
        encoding = '원핫 인코딩'
    
    # (4) 스케일링 여부
    scaling = '필요' if feature_type == '수치형' else '불필요'
    
    # (5) 상관관계 계산 (수치형만)
    corr = np.nan
    if dtype in ['int64', 'float64']:
        corr = train[[col, target_col]].corr().iloc[0,1]
    
    # (6) 결과 저장
    summary.append({
        '컬럼명': col,
        'dtype': dtype,
        '고유값 수': nunique,
        '결측값 수': missing,
        '결측률 (%)': missing_ratio,
        '유형': feature_type,
        '순서형 여부': ordinal,
        '인코딩 방식': encoding,
        '스케일링 여부': scaling,
        'SalePrice 상관관계': corr
    })

# 4. 데이터프레임화
summary_df = pd.DataFrame(summary)

# 5. Excel로 저장
output_path = "../data/description/feature_summary.xlsx"
summary_df.to_excel(output_path, index=False)
print("✅ 저장 완료:", output_path)


✅ 저장 완료: ../data/description/feature_summary.xlsx


In [14]:
# 컬럼의 고유값 종류 확인
print("BsmtFinType1 고유값:")
print(df['BsmtFinType1'].value_counts(dropna=False))

print("\nBsmtFinType2 고유값:")
print(df['BsmtFinType2'].value_counts(dropna=False))


BsmtFinType1 고유값:
BsmtFinType1
Unf    430
GLQ    418
ALQ    220
BLQ    148
Rec    133
LwQ     74
NaN     37
Name: count, dtype: int64

BsmtFinType2 고유값:
BsmtFinType2
Unf    1256
Rec      54
LwQ      46
NaN      38
BLQ      33
ALQ      19
GLQ      14
Name: count, dtype: int64


In [16]:
# ID 따로 보관
train_id = train['Id'].copy()
test_id = test['Id'].copy()

# ID 제거
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)